# Задание 1

Таблица представлена следующими полями:
1) key
2) id
3) phone
4) mail

Тестовые данные:

1;12345;89997776655; test@mail.ru

2;54321;87778885566; two@mail.ru

3;98765; 87776664577;three@mail

4; 66678;87778885566; four@mail.ru

5; 34567; 84547895566; four@mail.ru

6; 34567; 89087545678; five@mail.ru

На основании заданного поля (это может быть id, phone, mail) получить все "связанные данные"

Например:
если задать поиск по условию phone = 87778885566;

Результат должен быть следующим:

2;54321;87778885566; two@mail.ru

4; 66678;87778885566; four@mail.ru

5; 34567; 84547895566; four@mail.ru

6; 34567; 89087545678; five@mail.ru

In [25]:
# Импортируем библиотеку pandas для работы с датафреймами
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Рассмотрим два варианта:
1) датафрейм передается в тело функции и он изменяемый (например, данные из csv, сервера)

2) функция с подсказками для пользователя

В данном случае задаем датафрейм и передаем в тело функции в качестве входного параметра:

In [2]:
date = pd.DataFrame({'id': [12345, 54321, 98765, 66678, 34567, 34567],
                     'phone': [89997776655, 87778885566, 87776664577, 87778885566, 84547895566, 89087545678],
                     'mail': ['test@mail.ru', 'two@mail.ru', 'three@mail', 'four@mail.ru', 'four@mail.ru', 'five@mail.ru']},
                    index = {i for i in range(1, 7)})

Параметры col_name и value соответственно - название колонки и ее значение

In [7]:
def find_chain(date, col_name, value):
    """from date gets all chained data
       with input column name for searching and its value"""
#   Получаем датафрейм со строками, которые по первому поиску нашли
    clean_df = date.loc[date[col_name] == value]
    counter = 1
#     счетчик нужен для того, чтобы определить, что совпадений по поиску больше нет
    while counter != 0:
        counter = 0
        array = {}
        fl = pd.DataFrame()
        new = pd.DataFrame()
#        занесем значения из колонок датафрейма в словари,
#       где ключами являются названия колонок
        for column in clean_df:
            array[column] = clean_df[column].unique().tolist()
#       также создаем список индексов, уже попавших в датафрейм в первом приближении,
#       чтобы не найти их снова
        out_index = clean_df.index.values.tolist()
#     перебирая для всех ключей словаря значения в словаре, находим строки
#     в которых есть совпадения с текущими значениями
        for key, value in array.items():
            for elem in value:
                fl = fl.append(date.loc[date[key] == elem])
#     чтобы не добавить в датафрейм копии строк, учтем индексы уже добавленных
        for i in fl.index:
            if i not in out_index:
                new = new.append(fl.loc[[i]])
#     в случае, если найдена строка с еще не добавленным индексом, счетчик дает +1, чтобы цикл начался снова
#     если новых строк не найдено, то счетчик остается равен 0, цикл while завершается
                counter += 1
#     пополняем датафрейм
        clean_df = clean_df.append(new)
    return clean_df

In [26]:
# Как видим, работает
find_chain(date, 'phone', 87778885566)

,id,phone,mail
2,54321,87778885566,two@mail.ru
4,66678,87778885566,four@mail.ru
5,34567,84547895566,four@mail.ru
6,34567,89087545678,five@mail.ru


### Функция ниже для использования с возможностью подсказок пользователю, также работает

In [28]:
def search(date):
    col = str(input('Введите поле для поиска:'))
    while col not in date.columns:
        print(f'Доступные названия столбцов: {date.columns.values.tolist()}')
        col = str(input('Введите название колонки для поиска из доступных:'))     
    value = input('Введите значение для фильтрации:')
    while value not in [str(i) for i in date[f'{col}'].values]:
        print(f'Доступные значения колонок: {set(date[f"{col}"])}')
        value = input('Введите доступное значение для фильтрации:')
# Пока введенный параметр не совпадет с одним из значений в колонке, просим ввести 
# значение еще раз с подсказкой (перевод в строковый тип для сравнения чисел в таблице
# которые представлены в формате int, в функции ниже представлен аналогичный вариант 
# проверки вхождения элемента, но при сравнении строковых значений с числовыми
# возникает конфликт между numpy и pandas)
    try:
        value = int(value)
    except:
        value = value
    clean_df = date.loc[date[col] == value]
    counter = 1
    while counter != 0:
        counter = 0
        array = {}
        fl = pd.DataFrame()
        new = pd.DataFrame()
        for coln in date:
            array[coln] = clean_df[coln].unique().tolist()
        out_index = clean_df.index.values.tolist()
        for key, value in array.items():
            for elem in value:
                fl = fl.append(date.loc[date[key] == elem])
        for i in fl.index:
            if i not in out_index:
                new = new.append(fl.loc[[i]])
                counter += 1
        clean_df = clean_df.append(new)
    return clean_df

In [31]:
search(date)

Введите поле для поиска:id
Введите значение для фильтрации:34567


,id,phone,mail
5,34567,84547895566,four@mail.ru
6,34567,89087545678,five@mail.ru
4,66678,87778885566,four@mail.ru
2,54321,87778885566,two@mail.ru


#  Задание 2

Во вложении файл (данные полностью сгенерированы).
По каждому пользователю (столбец ID) посчитать сколько раз были выполнены следующие условия:
- был открыт новый договор (столбцы договор и дата начала) в один день с закрытием предыдущего договора (столбец дата завершения (факт))
- клиент по данному новому договору воспользовался новой сейфовой ячейкой (столбец сейф), но в том же офисе (столбец офис)

Далее используется понятие ключевое действие, которое включает в себя выполнение двух условий, представленных выше, т.е. клиент, совершивший ключевое действие:
- открыл новый договор в один день с закрытием предыдущего
- по данному новому договору воспользовался новой сейфовой ячейкой в том же офисе

In [36]:
# получим датафрейм из таблицы в excel и проанализируем колонки 
df = pd.read_excel('data.xlsx')

In [37]:
df.columns

Index(['Офис', 'Договор', 'Дата начала', 'Дата завершения (факт)', 'Сейф',
       'ID'],
      dtype='object')

In [38]:
df.dtypes

Офис                               int64
Договор                           object
Дата начала               datetime64[ns]
Дата завершения (факт)    datetime64[ns]
Сейф                               int64
ID                                 int64
dtype: object

Даты представлены в формате datetime, что хорошо для проведения анализа

In [39]:
# Название колонок переименуем для удобства, а также изменим порядок столбцов
df = df.rename(columns={'Офис': 'office', 'Договор': 'contract', 'Дата начала': 'start_date', 'Дата завершения (факт)': 'end_date', 'Сейф': 'safe', 'ID': "client_id"})
df = df.reindex(columns=["client_id", 'contract', 'start_date', 'end_date', 'safe', 'office'])

Посмотрим распеределение номеров договора в датафрейме

In [40]:
df.contract.value_counts()

9049-2925-000920599    6
9049-9779-000922524    5
9049-9779-000924929    5
5992-492-000092249     5
5992-492-000092222     5
                      ..
9547-292-000220254     1
9547-292-000220445     1
9547-292-000222454     1
9547-292-000222740     1
9070-75-000055429      1
Name: contract, Length: 44730, dtype: int64

Посмотрим, как выглядят строки с одинаковыми номерами договора

In [41]:
df.query('contract == "9049-9779-000922524"')

,client_id,contract,start_date,end_date,safe,office
39132,2242464644444477780,9049-9779-000922524,2022-02-28,NaT,885,8888
39136,2242464644444477780,9049-9779-000922524,2022-02-28,NaT,894,8888
39157,2242442727904867280,9049-9779-000922524,2022-02-28,NaT,447,8888
39274,2240277907472897740,9049-9779-000922524,2022-03-01,2022-03-16,531,8888
39280,2242428420779872449,9049-9779-000922524,2022-03-01,NaT,816,8888


In [42]:
df.query('contract == "9049-2925-000920599"')

,client_id,contract,start_date,end_date,safe,office
35012,2242424467424900449,9049-2925-000920599,2022-02-25,NaT,453,2826
35015,2242424467424900449,9049-2925-000920599,2022-02-25,NaT,457,2826
35026,222862424726276444,9049-2925-000920599,2022-02-25,NaT,471,2826
35027,222862424726276444,9049-2925-000920599,2022-02-25,NaT,473,2826
35029,2242487288607699269,9049-2925-000920599,2022-02-25,NaT,571,2826
35030,2242479249807777240,9049-2925-000920599,2022-02-25,NaT,65,2826


Как видим, строки дублируются при открытии на один договор нескольких сейфов, поэтому исключим их из датафрейма с учетом того, что клиенты с разными id могут заключать договора с одним номером

In [43]:
contract_dropped_df = df.drop_duplicates(subset=['client_id', 'contract'])

In [44]:
# датафрейм с id клиента с номером договора и началом его действия
df_1 = contract_dropped_df[['client_id', 'start_date', 'contract', 'safe', 'office']].rename(columns={'contract': 'new_contract', 'safe': 'new_safe', 'office': 'new_office'})
# датафрейм с id клиента, номером договора и временем его закрытием
df_2 = contract_dropped_df[['client_id', 'end_date', 'contract', 'safe', 'office']].rename(columns={'contract' : 'ended_contract', 'safe': 'old_safe', 'office': 'old_office'})
# Объединяем датафреймы по полям id клиента и времени окончания и начала договоров, а также столбцу офис
Full_df = df_2.merge(df_1, how='inner', left_on=['client_id', 'end_date', 'old_office'], right_on=['client_id', 'start_date', 'new_office'])

In [45]:
# В датафрейм попали договора, которые были закрыты днем начала, уберем их
Full_df = Full_df.query('new_contract != ended_contract and old_safe != new_safe')

In [46]:
# Второе условие требует, чтобы новый договор был заключен с новой сейфовой ячейкой
Full_df = Full_df.query('old_safe != new_safe')

In [47]:
# Посчитаем число событий, когда новый договор заключался в день закрытия предыдущего договора
Result_df = Full_df.groupby('client_id', as_index=False)\
                   .agg({'new_contract': 'count'})\
                   .rename(columns={'new_contract': 'number_of_actions'})\
                   .sort_values('number_of_actions', ascending=False)\
                   .reset_index(drop=True)

In [48]:
# Таблица id клиентов с выполненным ключевым действием
Result_df

,client_id,number_of_actions
0,224247202274274262,52
1,2240224227944820994,10
2,2240226498729726664,4
3,224026268462298777,4
4,2240272076889247040,3
...,...,...
327,2229424707722694060,1
328,2229424224904777942,1
329,2229406648448474649,1
330,2229044486222622494,1


In [49]:
# Число ключевых событий
Full_df.shape[0]

419

In [50]:
# Распределение клиентов по ключевому действию
Result_df.number_of_actions.value_counts()

1     309
2      17
4       2
3       2
52      1
10      1
Name: number_of_actions, dtype: int64

Клиент с наибольшим числом ключевых действий:

In [51]:
Max_client = Result_df.loc[Result_df.number_of_actions.idxmax()].client_id

Получили, что два клиента намного больше других совершили ключевое действие, посмотрим на исходный датафрейм с данными client id

In [52]:
df.query('client_id == @Max_client')

,client_id,contract,start_date,end_date,safe,office
39209,224247202274274262,9049-9779-000922045,2022-02-28,NaT,554,8888
39210,224247202274274262,9049-9779-000922042,2022-02-28,NaT,555,8888
39212,224247202274274262,9049-9779-000922045,2022-02-28,NaT,560,8888
39213,224247202274274262,9049-9779-000922052,2022-02-28,NaT,561,8888
39215,224247202274274262,9049-9779-000922055,2022-02-28,NaT,566,8888
39216,224247202274274262,9049-9779-000922052,2022-02-28,NaT,567,8888
39218,224247202274274262,9049-9779-000922059,2022-02-28,NaT,572,8888
39219,224247202274274262,9049-9779-000922074,2022-02-28,NaT,573,8888
39220,224247202274274262,9049-9779-000922092,2022-02-28,NaT,578,8888
39221,224247202274274262,9049-9779-000922095,2022-02-28,NaT,579,8888


Так как клиент совершил в один день 14 открытий договора, 4 из которых закрыл тем же днем, при их сравнении в таблице происходит пересечение, т.е. каждый закрытый договор с каждым открытым в тот же день и в том же офисе

То есть 4 * 13 = 52

Требуемые в задании условия выполняются в данном случае, поэтому результатом можно считать данный датафрейм:

In [53]:
Result_df

,client_id,number_of_actions
0,224247202274274262,52
1,2240224227944820994,10
2,2240226498729726664,4
3,224026268462298777,4
4,2240272076889247040,3
...,...,...
327,2229424707722694060,1
328,2229424224904777942,1
329,2229406648448474649,1
330,2229044486222622494,1


## Однако, если принять допущение, что заключенные в один день договора и закрытые также в один день считать за одно событие, то получим:

In [54]:
without_duplicates = df.drop_duplicates(subset=['client_id', 'start_date', 'end_date'])

In [55]:
without_duplicates.query('client_id == @Max_client')

,client_id,contract,start_date,end_date,safe,office
39209,224247202274274262,9049-9779-000922045,2022-02-28,NaT,554,8888
39224,224247202274274262,9049-9779-000922207,2022-02-28,2022-02-28,590,8888


14 строк для клиента с 52 ключевыми действиями теперь дает только 2, далее выполняем те же действия, что и ранее:

In [56]:
# датафрейм с id клиента с номером договора, началом его действия, сейфом и офисом
df_3 = without_duplicates[['client_id', 'start_date', 'contract', 'safe', 'office']].rename(columns={'contract': 'new_contract', 'safe': 'new_safe', 'office': 'new_office'})
# датафрейм с id клиента, номером договора, временем его закрытием, сейфом и офисом
df_4 = without_duplicates[['client_id', 'end_date', 'contract', 'safe', 'office']].rename(columns={'contract' : 'ended_contract', 'safe': 'old_safe', 'office': 'old_office'})
# Объединяем датафреймы по полям id клиента и времени окончания и начала договоров, а также столбцу офис
clean_df = df_4.merge(df_3, how='inner', left_on=['client_id', 'end_date', 'old_office'], right_on=['client_id', 'start_date', 'new_office'])

In [57]:
# Убираем договора, которые пересеклись сами с собой, а также у которых не изменилась ячейка
clean_df = clean_df.query('new_contract != ended_contract and old_safe != new_safe')

In [58]:
# выполняем группировку по id клиента, считаем число строк и сортируем фрейм по убыванию
Without_duplicates_df = (clean_df.groupby('client_id', as_index=False)
                             .agg({'new_contract': 'count'}).rename(columns={'new_contract': 'number_of_actions'})
                             .sort_values('number_of_actions', ascending=False).reset_index(drop=True))

Without_duplicates_df

,client_id,number_of_actions
0,2240272076889247040,3
1,2242464644444477780,3
2,2240799777744442226,2
3,2240497722677927672,2
4,2240224227944820994,2
...,...,...
325,2229406648448474649,1
326,2229044486222622494,1
327,2229029678908270292,1
328,2229028720904280804,1


In [59]:
Without_duplicates_df.number_of_actions.value_counts()

1    320
2      8
3      2
Name: number_of_actions, dtype: int64

In [60]:
clean_df.shape[0]

342

In [61]:
# Клиент, у которого было заключено 14 договоров в один день, теперь имеет только одно ключевое действие
clean_df.query('client_id == @Max_client')

,client_id,end_date,ended_contract,old_safe,old_office,start_date,new_contract,new_safe,new_office
460,224247202274274262,2022-02-28,9049-9779-000922207,590,8888,2022-02-28,9049-9779-000922045,554,8888


### Таким образом, в качестве ответа на второй вопрос о ключевых действиях в разбивке по каждому пользователю даю два решения:
### 1) Соответствующий условиям задания
### 2) Также соответствующий условиям задания, но с оговоркой, что заключенные в 1 день договора у одного клиента и закрытые также в 1 день, считать одним событием.
### Какой из двух итоговых датафреймов использовать, зависит от того, на какие вопросы необходимо ответить в дальнейшем, например, если необходимо посмотреть, сколько операций за один день совершил клиент с учетом текущих условий, то используем 1-ый, если требуется ответить на вопрос о распределении ключевого действия между всеми клиентами, то второй, так как он исключает выбросы, связанные с однотипными операциями в день

In [62]:
# согласно условий задания
Result_df

,client_id,number_of_actions
0,224247202274274262,52
1,2240224227944820994,10
2,2240226498729726664,4
3,224026268462298777,4
4,2240272076889247040,3
...,...,...
327,2229424707722694060,1
328,2229424224904777942,1
329,2229406648448474649,1
330,2229044486222622494,1


In [63]:
# с принятым допущением
Without_duplicates_df

,client_id,number_of_actions
0,2240272076889247040,3
1,2242464644444477780,3
2,2240799777744442226,2
3,2240497722677927672,2
4,2240224227944820994,2
...,...,...
325,2229406648448474649,1
326,2229044486222622494,1
327,2229029678908270292,1
328,2229028720904280804,1
